In [107]:
import pennylane as qml
import numpy as np
import jax
import matplotlib.pyplot as plt
from io import StringIO
import os
import cirq
from cirq import KakDecomposition
from typing import Tuple
import json
# Configure JAX
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")
jnp = jax.numpy


In [108]:
save = False

In [109]:
def ensure_directory_exists(directory):
    """Ensure that the specified directory exists, create it if it does not."""
    if not os.path.exists(directory):
        os.makedirs(directory)

def save_params(params, filename):
    """Save parameters to a file using JSON."""
    # Convert NumPy arrays to lists
    params_serializable = np.array(params).tolist()
    with open(filename, 'w') as f:
        json.dump(params_serializable, f)

def load_params(filename):
    """Load parameters from a file using JSON."""
    with open(filename, 'r') as f:
        params_serializable = json.load(f)
    # Convert lists back to NumPy arrays
    return jnp.array(params_serializable)



In [110]:
def get_initial_params(params_file):
    """Get initial parameters from the file if it exists, otherwise generate new ones."""
    if save and os.path.exists(params_file):
        # Load parameters if they exist
        return load_params(params_file)
    else:
        # Generate random parameters and save them
        init_params = jnp.array(np.random.randn(15))
        if save:
            save_params(init_params, params_file)
        return init_params


In [111]:
def create_matrix(params, num_wires):
    """Create a SU(4) unitary matrix from the given parameters."""
    return qml.SpecialUnitary.compute_matrix(theta = params, num_wires=num_wires)



In [112]:
def perform_kak_decomposition(gate, nqubits=2): 
    
    # Prepare Cirq circuit
    qubits = cirq.LineQubit.range(nqubits) 

    # Get the indices of the target gate
    # targets = gate.get_target_index_list() 

    cirq_gates = []
    
    gate = cirq.MatrixGate(gate).on(*qubits)

    # Perform KAK decomposition using Cirq
    kak_operations = cirq.kak_decomposition(gate) 
    
    # Get the tuple of coefficients(X,Y,Z)
    coef = kak_operations.interaction_coefficients
    
    # Get the tuple of gates before and after KAK decomposition
    initial_operations = kak_operations.single_qubit_operations_before 
    final_operations = kak_operations.single_qubit_operations_after 

    # Add initial operations to the list of Cirq gates
    cirq_gates.append(cirq.MatrixGate(initial_operations[0])(qubits[0]))
    cirq_gates.append(cirq.MatrixGate(initial_operations[1])(qubits[1]))
    
    # Add middle operations (XX+YY+ZZ) to the list of Cirq gates
    cirq_gates.append(coef[0])
    cirq_gates.append(coef[1])
    cirq_gates.append(coef[2])
    
    # Add final operations to the list of Cirq gates
    cirq_gates.append(cirq.MatrixGate(final_operations[0])(qubits[0]))
    cirq_gates.append(cirq.MatrixGate(final_operations[1])(qubits[1]))

    return cirq_gates

In [113]:
# def get_kak_vector(U):
#    qubits = [cirq.LineQubit(0), cirq.LineQubit(1)]
#    cirq.MatrixGate(U).on(*qubits)
#    return cirq.kak_vector(U)

In [114]:
def euler_decomposition(cirq_gate, nqubits=2):
    """
    Decompose a given single qubit cirq gate into a list of gates 
    using euler decomposition.
    
    Args:
        cirq_gate (cirq.ops.gate_operation.GateOperation) : Single qubit cirq gate to decompose
        nqubits (int) : Number of qubits in the quantum circuit

    Returns:
        cirq_gates (list) : List of decomposed Cirq gates
    """
    cirq_gates = []
    
    # Create a cirq circuit and append the gate
    qc = cirq.Circuit()
    qc.append(cirq_gate)
    
    # Get the unitary matrix of the input gate
    unitary_matrix = qc.unitary()
    
    # Prepare Cirq circuit
    circuit = cirq.LineQubit.range(nqubits)

    # Get the target qubit
    target_qubit = int(cirq_gate.qubits[0])
    
    # Perform Euler decomposition
    decomposition_angles = cirq.deconstruct_single_qubit_matrix_into_angles(unitary_matrix)
    
    # Add decomposed gates to the list
    cirq_gates.append(decomposition_angles[0])
    cirq_gates.append(decomposition_angles[1])
    cirq_gates.append(decomposition_angles[2])
    
    return cirq_gates

In [115]:
init_params1 = []
init_params2 = []
def run():
    directory = 'data/mapper/'
    params_file = os.path.join(directory, 'init_params.json')
    # Ensure the directory exists
    ensure_directory_exists(directory)

    # Get the initial parameters
    init_params1 = get_initial_params(params_file)
    # print(init_params1)
    # Create the SU(4) unitary matrix
    unitary = create_matrix(init_params1, 2)
    # Perform the KAK decomposition
    kak_operations = perform_kak_decomposition(unitary)
    # Perform Euler decomposition for initial and final operations separately
    initial_op0 = euler_decomposition(kak_operations[0])
    initial_op1 = euler_decomposition(kak_operations[1])
    final_op0 = euler_decomposition(kak_operations[5])
    final_op1 = euler_decomposition(kak_operations[6])
    #init_params2 = initial_op0+initial_op1+kak_operations[2:5]+final_op0+final_op1
    init_params2 = initial_op1+initial_op0+kak_operations[2:5][::-1]+final_op1+final_op0
    # print("params", init_params2 )
    return init_params1, init_params2
    #print("kak vector", get_kak_vector(unitary))
    # def __str__(self) -> str:
    #     xx = self.interaction_coefficients[0] * 4 / np.pi
    #     yy = self.interaction_coefficients[1] * 4 / np.pi
    #     zz = self.interaction_coefficients[2] * 4 / np.pi
    #     before0 = axis_angle(self.single_qubit_operations_before[0])
    #     before1 = axis_angle(self.single_qubit_operations_before[1])
    #     after0 = axis_angle(self.single_qubit_operations_after[0])
    #     after1 = axis_angle(self.single_qubit_operations_after[1])
    #     return (
    #         'KAK {\n'
    #         f'    xyz*(4/π): {xx:.3g}, {yy:.3g}, {zz:.3g}\n'
    #         f'    before: ({before0}) ⊗ ({before1})\n'
    #         f'    after: ({after0}) ⊗ ({after1})\n'
    #         '}'
    #     )

    # before0 = cirq.axis_angle(kak_decomp.single_qubit_operations_before[0])
    # before1 = cirq.axis_angle(kak_decomp.single_qubit_operations_before[1])
    # after0 = cirq.axis_angle(kak_decomp.single_qubit_operations_after[0])
    # after1 = cirq.axis_angle(kak_decomp.single_qubit_operations_after[1])
    # xx = kak_decomp.interaction_coefficients[0] * 4 / np.pi
    # yy = kak_decomp.interaction_coefficients[1] * 4 / np.pi
    # zz = kak_decomp.interaction_coefficients[2] * 4 / np.pi
    # print("A0", type(kak_decomp.single_qubit_operations_before[0]))
    # print("Euler Decomposition", euler_decomposition(cirq.MatrixGate(kak_decomp.single_qubit_operations_before[0]), 2))
    # print("KAK Decomposition Parameters:")
    # print(f"{before0}, {before1}, {xx}, {yy}, {zz}, {after0}, {after1}")
    # # print("a0:", a0)
    # print("a1:", a1)
    # print("B:", B)
    # print("b0:", b0)
    # print("b1:", b1)
    # print("Gamma:", gamma)



In [116]:
from typing import List, Protocol, Callable, Tuple
from typing import List, Tuple, Callable


# Configure JAX
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")
jnp = jax.numpy

def create_transverse_hamiltonian(num_wires: int, J: float = 1.0, h: float = 0.5):
    coeffs = []
    obs = []

    # ZZ interactions
    for i in range(num_wires):
        for j in range(i + 1, num_wires):
            coeffs.append(-J)
            obs.append(qml.PauliZ(i) @ qml.PauliZ(j))

    # X interactions
    for i in range(num_wires):
        coeffs.append(-h)
        obs.append(qml.PauliX(i))

    hamiltonian = qml.Hamiltonian(coeffs, obs)
    e_min = min(qml.eigvals(hamiltonian))
    e_max = max(qml.eigvals(hamiltonian))
    # hamiltonian = qml.SparseHamiltonian(hamiltonian.sparse_matrix(), range(num_wires))
    return hamiltonian, e_min, e_max

def create_heisenberg_hamiltonian(num_wires: int, J: float = 1.0):
    coeffs = []
    obs = []

    # Heisenberg interactions (XX + YY + ZZ)
    for i in range(num_wires):
        for j in range(i + 1, num_wires):
            coeffs.extend([-J, -J, -J])
            obs.extend([qml.PauliX(i) @ qml.PauliX(j), qml.PauliY(i) @ qml.PauliY(j), qml.PauliZ(i) @ qml.PauliZ(j)])

    hamiltonian = qml.Hamiltonian(coeffs, obs)
    e_min = min(qml.eigvals(hamiltonian))
    e_max = max(qml.eigvals(hamiltonian))
    # hamiltonian = qml.SparseHamiltonian(hamiltonian.sparse_matrix(), range(num_wires))
    return hamiltonian, e_min, e_max

def create_longitudinal_ising_hamiltonian(num_wires: int, J: float = 1.0, hx: float = 0.5, hz: float = 0.5):
    coeffs = []
    obs = []

    # ZZ interactions
    for i in range(num_wires):
        for j in range(i + 1, num_wires):
            coeffs.append(-J)
            obs.append(qml.PauliZ(i) @ qml.PauliZ(j))

    # X interactions (transverse field)
    for i in range(num_wires):
        coeffs.append(-hx)
        obs.append(qml.PauliX(i))
    
    # Z interactions (longitudinal field)
    for i in range(num_wires):
        coeffs.append(-hz)
        obs.append(qml.PauliZ(i))

    hamiltonian = qml.Hamiltonian(coeffs, obs)
    e_min = min(qml.eigvals(hamiltonian))
    e_max = max(qml.eigvals(hamiltonian))
    # hamiltonian = qml.SparseHamiltonian(hamiltonian.sparse_matrix(), range(num_wires))

    return hamiltonian, e_min, e_max

In [117]:

class Operation(Protocol):
    def apply(self, params: np.ndarray, wires: List[int]) -> None:
        ...

class TwoQubitDecomp:
    def apply(self, params: np.ndarray, wires: List[int]) -> None:
        """Implement an arbitrary SU(4) gate on two qubits
        using the decomposition from Theorem 5 in
        https://arxiv.org/pdf/quant-ph/0308006.pdf"""
        i, j = wires
        qml.RZ(params[0], wires=i)
        qml.RY(params[1], wires=i)
        qml.RZ(params[2], wires=i)

        qml.RZ(params[3], wires=j)
        qml.RY(params[4], wires=j)
        qml.RZ(params[5], wires=j)

        qml.CNOT(wires=[j, i])
        qml.RZ(params[6], wires=i)
        qml.RY(params[7], wires=j)
        qml.CNOT(wires=[i, j])
        qml.RY(params[8], wires=j)
        qml.CNOT(wires=[j, i])

        qml.RZ(params[9], wires=i)
        qml.RY(params[10], wires=i)
        qml.RZ(params[11], wires=i)

        qml.RZ(params[12], wires=j)
        qml.RY(params[13], wires=j)
        qml.RZ(params[14], wires=j)

class PauliRotSequence:
    def apply(self, params: np.ndarray, wires: List[int]) -> None:
        qml.ArbitraryUnitary(params, wires=wires)

class SpecialUnitaryGate:
    def apply(self, params: np.ndarray, wires: List[int]) -> None:
        qml.SpecialUnitary(params, wires=wires)

# Initialize parameters
num_wires = 6
loc = 2
learning_rate = 1e-4
num_steps = 1000
rebuild = False
# init_params = jax.numpy.array(init_params)
# # Define a function that returns a default value for the innermost dictionary
# def nested_dict():
#     return defaultdict(lambda: defaultdict(lambda: [None, None]))

# Initialize the top-level defaultdict
energies = {}

dev = qml.device("default.qubit", wires=num_wires)

def circuit(params, operation: Operation, hamiltonian):
        operation.apply(params, [0,1])
        return qml.expval(hamiltonian)


#  circuitLauncher
def launch_circuit(hamiltonian_func: Callable[[], Tuple[qml.Hamiltonian, float]]):
    hamiltonian, e_min, e_max = hamiltonian_func()
    def circuit_wrapped(params, operation: Operation):
        return circuit(params, operation, hamiltonian)

    qnode = qml.QNode(circuit_wrapped, dev, interface="jax")
    return qnode, e_min, e_max

In [122]:
if __name__ == "__main__":
    diffs = []
    for i in range(1):
      init_params1, init_params2 = run()
      print(init_params1)
      qnode, E_min, E_max = launch_circuit(lambda: create_transverse_hamiltonian(num_wires))
      cost1 = qnode(init_params1, SpecialUnitaryGate())
      init_params2[6] = 2* init_params2[6] - (np.pi/2)
      init_params2[7] =  (np.pi/2) - 2* init_params2[7]
      init_params2[8] = 2* init_params2[8] - (np.pi/2)
  
      cost2 = qnode(init_params2, TwoQubitDecomp())
      print("cost1", cost1)
      print("cost2", cost2)
      print(cost1 - cost2)
      #print(init_params1, init_params2)

[ 1.85362904e-01 -7.84122644e-01 -3.58636457e-01  3.83193599e-01
 -1.63783775e+00  5.44408764e-04  1.95265790e+00 -1.85235447e-01
  2.28833961e-01  5.02488906e-01 -1.86341364e+00  1.54873695e+00
  8.35772842e-01  7.42742753e-01  1.35733431e+00]
cost1 -5.33482000845444
cost2 -3.6219963596152116
-1.7128236488392288


/opt/homebrew/anaconda3/envs/pennylane/lib/python3.12/site-packages/pennylane/ops/functions/eigvals.py:120: UserWarning: For Hamiltonians, the eigenvalues will be computed numerically. This may be computationally intensive for a large number of wires. Consider using a sparse representation of the Hamiltonian with qml.SparseHamiltonian.
  warnings.warn(


In [123]:
unitary = create_matrix(init_params1, 2)

decomp = qml.ops.two_qubit_decomposition(unitary, wires=[0, 1])
print("Initial parameters (SU4):", init_params1)
print(decomp)
init_params2 = []
for gate in decomp:
    if hasattr(gate, 'parameters'):
        init_params2.extend([float(p) for p in gate.parameters])
print("Decomposed parameters:", init_params2)

cost1 = qnode(init_params1, SpecialUnitaryGate())

# init_params2[6] = 2 * init_params1[6] - (np.pi / 2)
# init_params2[7] = (np.pi / 2) - 2 * init_params1[7]
# init_params2[8] = 2 * init_params1[8] - (np.pi / 2)

cost2 = qnode(init_params2, TwoQubitDecomp())

print(f"Cost1: {cost1}")
print(f"Cost2: {cost2}")
print(f"Difference: {cost1 - cost2}")

Initial parameters (SU4): [ 1.85362904e-01 -7.84122644e-01 -3.58636457e-01  3.83193599e-01
 -1.63783775e+00  5.44408764e-04  1.95265790e+00 -1.85235447e-01
  2.28833961e-01  5.02488906e-01 -1.86341364e+00  1.54873695e+00
  8.35772842e-01  7.42742753e-01  1.35733431e+00]
[RZ(Array(11.97015254, dtype=float64), wires=[0]), RY(Array(2.10136286, dtype=float64), wires=[0]), RZ(Array(1.66796652, dtype=float64), wires=[0]), RZ(Array(1.15088188, dtype=float64), wires=[1]), RY(Array(0.58917511, dtype=float64), wires=[1]), RZ(Array(11.5313721, dtype=float64), wires=[1]), CNOT(wires=[1, 0]), RZ(-0.0925537900027627, wires=[0]), RY(-0.7368152274837142, wires=[1]), CNOT(wires=[0, 1]), RY(-1.2389989070887824, wires=[1]), CNOT(wires=[1, 0]), RZ(Array(11.9518672, dtype=float64), wires=[1]), RY(Array(1.81843634, dtype=float64), wires=[1]), RZ(Array(3.14404795, dtype=float64), wires=[1]), RZ(Array(0.36032973, dtype=float64), wires=[0]), RY(Array(1.12686268, dtype=float64), wires=[0]), RZ(Array(2.95705273,